# Imports

In [1]:
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, \
                         TrainingArguments, Trainer
from torch.optim import AdamW
from pprint import pprint

/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load and Process Dataset

We start by loading the conll data via the hugging face api and its load_dataset function.

In [2]:
raw_datasets = load_dataset("wnut_17")

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [4]:
raw_datasets["train"][0]["tokens"]

['@paulwalk',
 'It',
 "'s",
 'the',
 'view',
 'from',
 'where',
 'I',
 "'m",
 'living',
 'for',
 'two',
 'weeks',
 '.',
 'Empire',
 'State',
 'Building',
 '=',
 'ESB',
 '.',
 'Pretty',
 'bad',
 'storm',
 'here',
 'last',
 'evening',
 '.']

In [5]:
raw_datasets["train"][0]["ner_tags"]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 8,
 8,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

Followingly, we extract the named entitiy recognition features and the respective IOB labels.

In [6]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product'], id=None), length=-1, id=None)

We see that the train body features labels about corporations, creative-work, groups, locations, perople, and products.

In [7]:
label_names = ner_feature.feature.names
label_names

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

Based on this, we can express the decoded information and a respective sentence jointly:

In [8]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

@paulwalk It 's the view from where I 'm living for two weeks . Empire     State      Building   = ESB        . Pretty bad storm here last evening . 
O         O  O  O   O    O    O     O O  O      O   O   O     O B-location I-location I-location O B-location O O      O   O     O    O    O       O 


We will use a pretrained bert model to evaluate the contents of the WNUT dataset. This dataset features a lot of rare entities and thereby allows for testing models on largely unseen information.

In [9]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)

Next, we will give the tokens new labels which align with their new labels, which express their purpose numerically.

In [11]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [12]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [13]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

Having composed a function to tokenise and align the labels, we finally arrive at a preprocessed and tokenised dataset.

In [14]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

In [15]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Setting Evaluation

In [16]:
metric = evaluate.load("seqeval")

In [17]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [18]:
def evaluate_token_classification(logits, labels, entity_types):
    predictions = np.argmax(logits, axis=-1)
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    precision_dict = {}
    recall_dict = {}
    f1_dict = {}
    support_dict = {}
    
    for entity_type in entity_types:
        precision_dict[entity_type] = {}
        recall_dict[entity_type] = {}
        f1_dict[entity_type] = {}
        support_dict[entity_type] = {}
        b_true_labels_binary = [[l if f"B-{entity_type}" == l else 'O' for l in label] for label in true_labels]
        b_pred_labels_binary = [[l if f"B-{entity_type}" == l else 'O' for l in label] for label in predictions]
        
        b_metrics = metric.compute(predictions=b_pred_labels_binary, references=b_true_labels_binary)
        
        precision_dict[entity_type]['B-label'] = b_metrics[entity_type]['precision']
        recall_dict[entity_type]['B-label'] = b_metrics[entity_type]['recall']
        f1_dict[entity_type]['B-label'] = b_metrics[entity_type]['f1']
        support_dict[entity_type]['B-label'] = b_metrics[entity_type]['number']
        
        i_true_labels_binary = [[l if f"I-{entity_type}" == l else 'O' for l in label] for label in true_labels]
        i_pred_labels_binary = [[l if f"I-{entity_type}" == l else 'O' for l in label] for label in predictions]
    
        i_metrics = metric.compute(predictions=i_pred_labels_binary, references=i_true_labels_binary)
        
        precision_dict[entity_type]['I-label'] = i_metrics[entity_type]['precision']
        recall_dict[entity_type]['I-label'] = i_metrics[entity_type]['recall']
        f1_dict[entity_type]['I-label'] = i_metrics[entity_type]['f1']
        support_dict[entity_type]['I-label'] = i_metrics[entity_type]['number']

    entity_metrics = metric.compute(predictions=predictions, references=true_labels)
    
    for entity_type in entity_types:
        precision_dict[entity_type]['entity'] = entity_metrics[entity_type]['precision']
        recall_dict[entity_type]['entity'] = entity_metrics[entity_type]['recall']
        f1_dict[entity_type]['entity'] = entity_metrics[entity_type]['f1']
        support_dict[entity_type]['entity'] = entity_metrics[entity_type]['number']
        
    f1_scores_list = [f1_dict[entity_type]["entity"] for entity_type in f1_dict] 
    support_list = [support_dict[entity_type]["entity"] for entity_type in support_dict]
    weights_support_list = [support / sum(support_list) for support in support_list]
    
    final_dict = {
        "precision": precision_dict,
        "recall": recall_dict,
        "f1": f1_dict,
        "macro_f1": sum(f1_scores_list) / len(f1_scores_list),
        "micro_f1": sum([f1_score * support for f1_score, support in zip(f1_scores_list, weights_support_list)])
    }
    return final_dict

In [19]:
entity_types = ['corporation', 'creative-work', 'group', 'location', 'person', 'product']

# Fine-tune Model

In [20]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

### Baseline model

We will use the bert model with baseline parameters for our baseline model.

In [22]:
args_baseline = TrainingArguments("bert-finetuned-baseline")
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

trainer_baseline = Trainer(
    model=model,
    args=args_baseline,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

trainer_baseline.train()

test_pred_baseline = trainer_baseline.predict(test_dataset=tokenized_datasets["test"])
test_pred_baseline.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.118500
1000,0.042900


{'test_loss': 0.2598603665828705,
 'test_precision': 0.49262536873156343,
 'test_recall': 0.30954587581093607,
 'test_f1': 0.3801935116676152,
 'test_accuracy': 0.9336180904522613,
 'test_runtime': 101.5164,
 'test_samples_per_second': 12.678,
 'test_steps_per_second': 1.586}

In [60]:
macro_micro_baseline = evaluate_token_classification(logits=test_pred_baseline.predictions, \
                                                     labels=test_pred_baseline.label_ids, \
                                                     entity_types=entity_types)
pprint(macro_micro_baseline)

{'f1': {'corporation': {'B-label': 0.23999999999999996,
                        'I-label': 0.21487603305785122,
                        'entity': 0.19047619047619047},
        'creative-work': {'B-label': 0.37948717948717947,
                          'I-label': 0.31413612565445026,
                          'entity': 0.27488151658767773},
        'group': {'B-label': 0.270042194092827,
                  'I-label': 0.19883040935672516,
                  'entity': 0.2076923076923077},
        'location': {'B-label': 0.5886792452830187,
                     'I-label': 0.43113772455089816,
                     'entity': 0.4981949458483754},
        'person': {'B-label': 0.5947611710323574,
                   'I-label': 0.44353182751540043,
                   'entity': 0.5424739195230999},
        'product': {'B-label': 0.2650602409638554,
                    'I-label': 0.14689265536723164,
                    'entity': 0.13612565445026178}},
 'macro_f1': 0.30830742242965214,
 'micro_f1': 

Overall, the results are rather poor. Next, we will apply different optimisation approaches to improve the performance.

### AdamW Optimization

We begin by implementing the AdamW optimiser, which makes use of the general Adam optimisation approach and adds L2 regularisation via a decay in the parameter weights at each iteration. By implementing weight decay, we ensure that the model is more generaliseable and performs better on unseen data. The hyperparameters are again based on a a premade bert configuration.

In [50]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer1 = AdamW(model.parameters(), lr=2e-5)
args_adam1 = TrainingArguments(
    "bert-finetuned-adam1",
    evaluation_strategy="epoch",
)

trainer_adam1 = Trainer(
    model=model,
    args=args_adam1,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer1, None)
)

trainer_adam1.train()
test_pred_adam1 = trainer_adam1.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam1.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.161831,0.549470,0.372010,0.443652,0.917313
2,0.121300,0.180891,0.584437,0.422249,0.490278,0.921862
3,0.046900,0.176216,0.556028,0.468900,0.508761,0.925769


{'test_loss': 0.23555749654769897,
 'test_precision': 0.5247208931419458,
 'test_recall': 0.3049119555143652,
 'test_f1': 0.3856975381008207,
 'test_accuracy': 0.9348994974874372,
 'test_runtime': 9.7868,
 'test_samples_per_second': 131.504,
 'test_steps_per_second': 16.451}

In [61]:
macro_micro_adam1 = evaluate_token_classification(logits=test_pred_adam1.predictions, \
                                                  labels=test_pred_adam1.label_ids, \
                                                  entity_types=entity_types)
pprint(macro_micro_adam1)

{'f1': {'corporation': {'B-label': 0.23762376237623764,
                        'I-label': 0.2857142857142857,
                        'entity': 0.18333333333333332},
        'creative-work': {'B-label': 0.36756756756756753,
                          'I-label': 0.31016042780748665,
                          'entity': 0.2772277227722772},
        'group': {'B-label': 0.20960698689956328,
                  'I-label': 0.17073170731707318,
                  'entity': 0.16260162601626016},
        'location': {'B-label': 0.5490196078431373,
                     'I-label': 0.4516129032258065,
                     'entity': 0.4866920152091255},
        'person': {'B-label': 0.6015267175572518,
                   'I-label': 0.46694214876033063,
                   'entity': 0.5744047619047619},
        'product': {'B-label': 0.28402366863905326,
                    'I-label': 0.14973262032085563,
                    'entity': 0.12807881773399013}},
 'macro_f1': 0.30205637949495806,
 'micro_f1':

We directly observe an improvement in precision and recall.

We continue by manually specifying a larger batch size. While this will speed up computation, it may also harm generalisability.

In [22]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer2 = AdamW(model.parameters(), lr=2e-5)
args_adam2 = TrainingArguments(
    "bert-finetuned-adam2",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
)

trainer_adam2 = Trainer(
    model=model,
    args=args_adam2,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer2, None)
)

trainer_adam2.train()
test_pred_adam2 = trainer_adam2.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam2.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.218350,0.412844,0.053828,0.095238,0.886379
2,No log,0.195233,0.510588,0.259569,0.344171,0.903238
3,No log,0.187051,0.521891,0.356459,0.423596,0.912550


/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.22683948278427124,
 'test_precision': 0.47680890538033394,
 'test_recall': 0.2381835032437442,
 'test_f1': 0.31767614338689737,
 'test_accuracy': 0.9293467336683418,
 'test_runtime': 136.6502,
 'test_samples_per_second': 9.418,
 'test_steps_per_second': 1.178}

In [23]:
macro_micro_adam2 = evaluate_token_classification(logits=test_pred_adam2.predictions, \
                                                  labels=test_pred_adam2.label_ids, \
                                                  entity_types=entity_types)
pprint(macro_micro_adam2)

/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'f1': {'corporation': {'B-label': 0.057971014492753624,
                        'I-label': 0.03571428571428572,
                        'entity': 0.028985507246376812},
        'creative-work': {'B-label': 0.0,
                          'I-label': 0.02564102564102564,
                          'entity': 0.08284023668639053},
        'group': {'B-label': 0.0790960451977401,
                  'I-label': 0.046875,
                  'entity': 0.07446808510638299},
        'location': {'B-label': 0.5035971223021581,
                     'I-label': 0.3602484472049689,
                     'entity': 0.42657342657342656},
        'person': {'B-label': 0.4680851063829786,
                   'I-label': 0.30268199233716475,
                   'entity': 0.5071633237822349},
        'product': {'B-label': 0.0,
                    'I-label': 0.029850746268656716,
                    'entity': 0.038461538461538464}},
 'macro_f1': 0.1930820196427251,
 'micro_f1': 0.2895340930523538,
 'precision': {'c

Indeed, the model took less time to run 3 epochs, however suffered in performance. Particularly recall and thus F1 were harmed.

Next, we increase the learning rate. This may also have a positive impact on computation time, while also acting as regularisation. Perhaps, the impact will not be as dire as with batch computation.

In [21]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer3 = AdamW(model.parameters(), lr=9e-5)
args_adam3 = TrainingArguments(
    "bert-finetuned-adam3",
    evaluation_strategy="epoch",
)

trainer_adam3 = Trainer(
    model=model,
    args=args_adam3,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer3, None)
)

trainer_adam3.train()
test_pred_adam3 = trainer_adam3.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam3.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.173663,0.555992,0.338517,0.420818,0.913192
2,0.102900,0.182855,0.624088,0.409091,0.494220,0.919026
3,0.037200,0.223293,0.596748,0.438995,0.505858,0.921327


{'test_loss': 0.2652566432952881,
 'test_precision': 0.5217391304347826,
 'test_recall': 0.3002780352177943,
 'test_f1': 0.38117647058823534,
 'test_accuracy': 0.932889447236181,
 'test_runtime': 88.0235,
 'test_samples_per_second': 14.621,
 'test_steps_per_second': 1.829}

In [22]:
macro_micro_adam3 = evaluate_token_classification(logits=test_pred_adam3.predictions, \
                                                  labels=test_pred_adam3.label_ids, \
                                                  entity_types=entity_types)
pprint(macro_micro_adam3)

{'f1': {'corporation': {'B-label': 0.15789473684210525,
                        'I-label': 0.1836734693877551,
                        'entity': 0.109375},
        'creative-work': {'B-label': 0.40625,
                          'I-label': 0.3191489361702128,
                          'entity': 0.3121951219512195},
        'group': {'B-label': 0.2882096069868996,
                  'I-label': 0.20118343195266272,
                  'entity': 0.2390438247011952},
        'location': {'B-label': 0.5783132530120482,
                     'I-label': 0.4383561643835617,
                     'entity': 0.5},
        'person': {'B-label': 0.5924812030075188,
                   'I-label': 0.4512195121951219,
                   'entity': 0.5364431486880467},
        'product': {'B-label': 0.15789473684210523,
                    'I-label': 0.10126582278481013,
                    'entity': 0.08045977011494251}},
 'macro_f1': 0.296252810909234,
 'micro_f1': 0.37659424465666363,
 'precision': {'corpor

The model actually has better precision than the baseline. However, recall still underperforms, leading to an overall lower F1. It appears that the true positives of the test data do not resemble the training data too well, making the recall worse.
We will try the same learning rate, including the increased batch size.


In [21]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer4 = AdamW(model.parameters(), lr=9e-5)
args_adam4 = TrainingArguments(
    "bert-finetuned-adam4",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
)

trainer_adam4 = Trainer(
    model=model,
    args=args_adam4,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer4, None)
)

trainer_adam4.train()
test_pred_adam4 = trainer_adam4.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam4.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.204903,0.550000,0.250000,0.343750,0.902810
2,No log,0.177497,0.562278,0.377990,0.452074,0.915547
3,No log,0.173086,0.524725,0.456938,0.488491,0.922451


/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.22278931736946106,
 'test_precision': 0.45241379310344826,
 'test_recall': 0.303985171455051,
 'test_f1': 0.3636363636363637,
 'test_accuracy': 0.9338442211055277,
 'test_runtime': 140.9482,
 'test_samples_per_second': 9.131,
 'test_steps_per_second': 1.142}

In [22]:
macro_micro_adam4 = evaluate_token_classification(logits=test_pred_adam4.predictions, \
                                                  labels=test_pred_adam4.label_ids, \
                                                  entity_types=entity_types)
pprint(macro_micro_adam4)

{'f1': {'corporation': {'B-label': 0.2580645161290323,
                        'I-label': 0.1769911504424779,
                        'entity': 0.1456953642384106},
        'creative-work': {'B-label': 0.36180904522613067,
                          'I-label': 0.25555555555555554,
                          'entity': 0.22535211267605632},
        'group': {'B-label': 0.29310344827586204,
                  'I-label': 0.1807909604519774,
                  'entity': 0.19696969696969696},
        'location': {'B-label': 0.6039215686274509,
                     'I-label': 0.43137254901960786,
                     'entity': 0.5037593984962406},
        'person': {'B-label': 0.5961820851688693,
                   'I-label': 0.46613545816733065,
                   'entity': 0.5436337625178828},
        'product': {'B-label': 0.32183908045977005,
                    'I-label': 0.11640211640211641,
                    'entity': 0.0947867298578199}},
 'macro_f1': 0.2850328441260179,
 'micro_f1': 0.

As it has proven challenging to improve the model, we will also try a lower learning rate than at baseline.

In [21]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer5 = AdamW(model.parameters(), lr=1e-5)
args_adam5 = TrainingArguments(
    "bert-finetuned-adam5",
    evaluation_strategy="epoch",
)

trainer_adam5 = Trainer(
    model=model,
    args=args_adam5,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer5, None)
)

trainer_adam5.train()
test_pred_adam5 = trainer_adam5.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam5.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.184176,0.580097,0.285885,0.383013,0.908108
2,0.145200,0.178682,0.597070,0.389952,0.471780,0.918598
3,0.063000,0.166287,0.571429,0.440191,0.497297,0.920792


/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.2242525964975357,
 'test_precision': 0.4936102236421725,
 'test_recall': 0.28637627432808155,
 'test_f1': 0.3624633431085044,
 'test_accuracy': 0.9333668341708543,
 'test_runtime': 89.0857,
 'test_samples_per_second': 14.447,
 'test_steps_per_second': 1.807}

In [22]:
macro_micro_adam5 = evaluate_token_classification(logits=test_pred_adam5.predictions, \
                                                  labels=test_pred_adam5.label_ids, \
                                                  entity_types=entity_types)
pprint(macro_micro_adam5)

{'f1': {'corporation': {'B-label': 0.2456140350877193,
                        'I-label': 0.2772277227722772,
                        'entity': 0.17777777777777776},
        'creative-work': {'B-label': 0.11688311688311688,
                          'I-label': 0.14893617021276598,
                          'entity': 0.17910447761194032},
        'group': {'B-label': 0.16494845360824745,
                  'I-label': 0.0979020979020979,
                  'entity': 0.13333333333333333},
        'location': {'B-label': 0.5681818181818182,
                     'I-label': 0.4645161290322581,
                     'entity': 0.48905109489051096},
        'person': {'B-label': 0.5885978428351311,
                   'I-label': 0.4630738522954092,
                   'entity': 0.5568513119533529},
        'product': {'B-label': 0.11267605633802817,
                    'I-label': 0.06382978723404256,
                    'entity': 0.07035175879396985}},
 'macro_f1': 0.26774495906014756,
 'micro_f1': 

The decrease in model performance is negligible.
Again, we also try to increase the batch size to 32.

In [21]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
optimizer6 = AdamW(model.parameters(), lr=1e-5)
args_adam6 = TrainingArguments(
    "bert-finetuned-adam6",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
)

trainer_adam6 = Trainer(
    model=model,
    args=args_adam6,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    optimizers=(optimizer6, None)
)

trainer_adam6.train()
test_pred_adam6 = trainer_adam6.predict(test_dataset=tokenized_datasets["test"])
test_pred_adam6.metrics

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.239804,0.000000,0.000000,0.000000,0.881670
2,No log,0.220329,0.340909,0.053828,0.092975,0.889323
3,No log,0.211092,0.492647,0.160287,0.241877,0.897458


/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/arsen/miniconda3/envs/text_min/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.230812668800354,
 'test_precision': 0.46616541353383456,
 'test_recall': 0.11492122335495829,
 'test_f1': 0.18438661710037177,
 'test_accuracy': 0.922713567839196,
 'test_runtime': 140.0568,
 'test_samples_per_second': 9.189,
 'test_steps_per_second': 1.15}

In [22]:
macro_micro_adam6 = evaluate_token_classification(logits=test_pred_adam6.predictions, \
                                                  labels=test_pred_adam6.label_ids, \
                                                  entity_types=entity_types)
pprint(macro_micro_adam6)

{'f1': {'corporation': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0},
        'creative-work': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0},
        'group': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0},
        'location': {'B-label': 0.05063291139240507,
                     'I-label': 0.06451612903225808,
                     'entity': 0.22119815668202766},
        'person': {'B-label': 0.2589641434262948,
                   'I-label': 0.21399176954732513,
                   'entity': 0.3333333333333333},
        'product': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0}},
 'macro_f1': 0.09242191500256015,
 'micro_f1': 0.16328055931631524,
 'precision': {'corporation': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0},
               'creative-work': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0},
               'group': {'B-label': 0.0, 'I-label': 0.0, 'entity': 0.0},
               'location': {'B-label': 0.5,
                            'I-label': 0.08333333333333333,
    

This model struggles a lot to correctly identify true positives from false negatives, leading to a terrible Recall and F1 score.